# An Improved Chemical Reaction Optimization Algorithm for Solving the Shortest Common Supersequence Problem

In [550]:
import random
import math
import copy
from typing import List,Tuple
import time
from random import uniform, randint, shuffle, sample

# MY IMCRO

## Molecule Class



In [551]:
class MoleCule:
    def __init__(self, structure: List[float] ,supersequence: List[int]) -> None:
        '''
            PE : Potential Energy (This is the number we focus on during the algorithm)

            structure : Molecule Structe store as 1d array with 2 elements generated randomly (For example: [0.3860863116352774 0.4017794415965995])

        '''
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)
        self.supersequence = copy.deepcopy(supersequence)

    def update(self) -> None:
        """
        This is called whenever a Operator is performed
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self) -> str:
        reVal = 'Structure'
        reVal +='[ '
        for i in self.structure:
            reVal+=str(i)+" "
        reVal +=']  Supersequence[ '
        for i in self.supersequence:
            reVal+=str(i)+" "
        reVal +=' ]'
        return reVal
    
    def getSuper(self):
        return self.supersequence

## I. Initialization

In [552]:
def insert_symbol(src_string,inserted_string,pos):
    return ''.join(src_string[:pos] + inserted_string + src_string[pos:])
# Given set of strings and population size for SCS problem
def supersequence_generate(set_of_strings):

    '''
        Make a copy of the set_of_strings parameter for maintaining the original 
        set
    '''
    copied_set_of_strings = copy.deepcopy(set_of_strings)
    supersequence = ''.join(copied_set_of_strings.pop(random.randint(0,len(set_of_strings)-1)))

    for i in range(len(copied_set_of_strings)):
        # print("i = ",i)
        counter = 0
        for j in copied_set_of_strings[i]:
            inserted_pos = random.randint(counter,len(supersequence))
            # print("j and counter and supersequence length and inserted index",j," ",counter," ",len(supersequence)," ",inserted_pos)
            if inserted_pos == len(supersequence) or j != supersequence[inserted_pos]:
                supersequence=insert_symbol(supersequence,j,inserted_pos)
            counter = inserted_pos +1
            # print(supersequence)

    return supersequence


def population_generation(pop_size,set_of_strings):
    l=[]
    for i in range(pop_size):
        l.append(supersequence_generate(set_of_strings))
    return l



### 1. Encoding sequence
- In each molecule, it also have 4 characters (a, c, g, t)
- So, it respresent this symbols to integer (0, 1, 2,3 )

In [553]:
def encoding_population(initial_population):
    '''
        Encoding base on the class of the data
    '''
    dict = {
        'a' : 0,
        'c' : 1,
        'g' : 2,
        't' : 3
    }
    l=[]
    for i in initial_population:
        k=''
        for j in i:
            k+=str(dict[j])
        l.append(k)
    return l

In [554]:
def createMolecule(string):
    l=[]
    for i in string:
        l.append(int(i))
    return MoleCule(structure=[random.random(), random.random()],supersequence=l)

In [555]:
def is_subsequence(sub, main):
    it = iter(main)
    return all(any(item == x for x in it) for item in sub)

def is_supersequence(seq):
    subsequences = [[0, 1, 2], [1, 0, 3], [2, 3, 3], [3, 2, 1]]
    for i in subsequences:
        if not is_subsequence(i,seq):
            return False
    return True

# Example usage
sequence = [ 0, 2, 1, 0, 2, 0, 1, 2, 3, 0, 0, 2]
result = is_supersequence(sequence)
print(f"The sequence {sequence} is a supersequence: {result}")


The sequence [0, 2, 1, 0, 2, 0, 1, 2, 3, 0, 0, 2] is a supersequence: False


In [556]:
def initialization(pop_size,set_of_strings) ->List[MoleCule]:
    initial_population =  population_generation(pop_size,set_of_strings)
    print("Population Generation:")
    print(initial_population)
    encoded_population = encoding_population(initial_population)
    print("Encodeding:")
    print(encoded_population)
    l=[]
    for i in encoded_population:
        l.append(createMolecule(i))
    return l

molecules=initialization(20,['acg', 'cat', 'gtt','tgc'])
# 0 1 2 | 1 0 3 | 2 3 3 | 3 2 1
for i in molecules:
    print(i)

Population Generation:
['catagtttcgc', 'gtactggccatt', 'gacatttcgtgc', 'ctgatgcctgat', 'tcagactctg', 'tgactcattg', 'caactggttgc', 'atgctcgattc', 'atgcatcggtct', 'gactcatgtgc', 'gtatccagtgc', 'cactgtgct', 'agtctagctcg', 'catgacgtgtc', 'catgtcgctgt', 'tagccgatgtt', 'tcacggctt', 'gtactcgtatgc', 'agtctcgatgc', 'tagccgttat']
Encodeding:
['10302333121', '230132211033', '201033312321', '132032113203', '3102013132', '3201310332', '10013223321', '03213120331', '032103122313', '20131032321', '23031102321', '101323213', '02313021312', '10320123231', '10323121323', '30211203233', '310122133', '230131230321', '02313120321', '3021123303']
Structure[ 0.37088007739726847 0.3286447079945495 ]  Supersequence[ 1 0 3 0 2 3 3 3 1 2 1  ]
Structure[ 0.6430142945382863 0.8639834609188377 ]  Supersequence[ 2 3 0 1 3 2 2 1 1 0 3 3  ]
Structure[ 0.6914270399246876 0.8020235143465118 ]  Supersequence[ 2 0 1 0 3 3 3 1 2 3 2 1  ]
Structure[ 0.4035307709124065 0.13953007464702027 ]  Supersequence[ 1 3 2 0 3 2 1 1 3 

## II. Operators Class
- OnWall Ineffective Colision
- Intermolecular Ineffective Colision
- Decomposition
-  Synthesis

In [572]:
class Operators():
    
    # OnWall Ineffective Colision
    def OnWall (self,molecule : List[int]) -> List[int]:
        '''
        Objective: 
        -  This focuses on instances of molecules colliding with container walls, resulting in structural transformations.
        - A "one-difference operator" is used to make a single change in the molecule's composition to achieve this.

        Input:
        - molecule (list): the input molecule and it represent by list

        Output:
        - The method returns a new list. 
        '''
        #Initial
        m = copy.deepcopy(molecule)
        v_low = 0
        v_upper = 3   
        i = random.randint(0, len(molecule)-1)
        j = random.randint(v_low, v_upper)


        if (molecule[i] + j <= v_upper):    
            m[i] = molecule[i] + j
        else:
            if(molecule[i] - j < 0):
                m[i] = 0
            else:
                m[i] = molecule[i] - j
        
        #Test     
        # print(i, j)
        return m

    # Intermolecular Ineffective Colision
    def Intermolecular(self,molecule1 : List[int], molecule2 : List[int]) -> Tuple[List[int],List[int]]:
        '''
        Objective: 
        -  The purpose is to introduce significant changes to enhance local search capabilities and prevent getting stuck in local optimization by promoting diversity.
        - A crossover operator is used in genetic or evolutionary algorithms for optimization. It selects two molecules from the population and uses a two-step mechanism to generate two new solutions.
        - It is a two step process: the first step is to crossover between two molecules, and the second step is to crossover inside the molecule itself

        Input:
        - molecule (list): the input molecule and it represent by list

        Output:
        - The method returns a tuple (m1, m2), where m1 and m2 are the two molecules and m1, m2 are also list. 
        '''
        #Initial 
        # Length of molecule
        # print(molecule1,molecule2)
        length1 = len(molecule1)
        length2 = len(molecule2)
    
        #Two new molecule in first crossover
        # Copy 2 molecules
        m1 = molecule1.copy()
        m2 = molecule2.copy()
        #Two new molecule in second crossover
        m11 = [0] * length1
        m22 = [0] * length2
        
        limit = min(length2, length1) 
        
        #Random numbers x1, x2 generation
        x1 = random.randint(0, limit-2)
        x2 = random.randint(x1+1, limit-1)

    
        # Randormly choose form molecule1 or molecule2
        # Crossover 1
        for i in range(0,limit):
            if (i<x1 or i>x2):  #if odd segments
                m1[i] = molecule1[i]
                m2[i] = molecule2[i]
            elif (i>=x1 and x2>=i): # if even segment
                m1[i] = molecule2[i]
                m2[i] = molecule1[i]
        
        # Crossover 2
        # Crossover 2 for molecule m1
        for j in range(0,length1):
            if (j < x1):  #if odd segments
                m11[length1 - x1 + j] = m1[j]
                
            elif (j>=x1 and x2>=j): # if even segment
                m11[length1 - x1 - x2 + j - 1] = m1[j]
            else:
                m11[j - x2-1] = m1[j]
        
        # Crossover 2 for molecule m2
        for j in range(0,length2):
            if (j < x1):  #if odd segments
                m22[length2 - x1 + j] = m2[j]
            elif (j>=x1 and x2>=j): # if even segment
                m22[length2 - x1 - x2 + j- 1] = m2[j]
            else:
                m22[j - x2-1] = m2[j]
        
        #Test
        
        # print(limit)
        # print(x1, x2)
        # print(m1)
        # print(m2)
        return m11,m22
    
    # Decomposition
    def Decomposition(self, molecule : List[int]) -> Tuple[List[int],List[int]]:
        '''
        Objective: 
        - The decomposition involves randomly selecting two numbers 'a' and 'b', and then splitting the input molecule into two new molecules, 'm1' and 'm2', based on the selected numbers. 
        - The negative number −a is used for shifting to the left a steps. 
        - The positive number j is used for shifting to the right j steps.

        Input:
        - molecule (list): the input molecule and it represent by list

        Output:
        - The method returns a tuple (m1, m2), where m1 and m2 are the two molecules and m1, m2 are also list. 
        '''
        # Step 1: Select two numbers a and b randomly
        a = random.randint(-len(molecule), 0)
        b = random.randint(0, len(molecule))

        # Initialize m1 and m2
        m1 = [0] * len(molecule)
        m2 = [0] * len(molecule)

        # Step 2: Decomposition of molecule into m1
        for i in range(len(molecule)):
            if i + 1 <= -a:
                m1[len(molecule) - (-a) + i] = molecule[i]
            else:
                m1[i - (-a)] = molecule[i]

        # Step 3: Decomposition of molecule into m2
        for j in range(len(molecule)):
            if j + 1 <= len(molecule) - b:
                m2[j + b] = molecule[j]
            else:
                m2[j - len(molecule) + b] = molecule[j]
                
        #Test
        # print(molecule)
        # print(a, b)
        
        return m1, m2

    # Synthesis
    def Synthesis(self, molecule1 : List[int], molecule2 : List[int])-> List[int]:
        """
        Objective:
        - Generates a new list by combining two input lists in a way that preserves the frequency of the symbols used in each input list.

        Input:
        - molecule1 (list): The first input list.
        - molecule2 (list): The second input list.

        Output:
        - The method returns a new list. 
        """
        # Generate dictionaries for the frequencies of the symbols used in each input list.
        array1 = {}
        for symbol in molecule1:
            if symbol not in array1:
                array1[symbol] = 0
            array1[symbol] += 1

        array2 = {}
        for symbol in molecule2:
            if symbol not in array2:
                array2[symbol] = 0
            array2[symbol] += 1

        # Initialize the output list.
        length1 = len(molecule1)
        length2 = len(molecule2)
        limit = min(length1, length2)
        
        if(length1 < length2):
            m_prime = molecule2.copy()
        else:
            m_prime = molecule1.copy()

        # Iterate over the symbols in the first input list.
        for i in range(limit):
            symbol1 = molecule1[i]
            symbol2 = molecule2[i]

            frequency_in_array1 = array1.get(symbol1, 0)
            frequency_in_array2 = array2.get(symbol2, 0)

            if frequency_in_array1 >= frequency_in_array2:
                m_prime[i] = symbol1
            # Otherwise, add the symbol from the second input list to the output list.
            else:
                m_prime[i] = symbol2
        #test
        
        # print(molecule1)
        # print(molecule2)
        # print(array1)
        # print(array2)
        
        return m_prime

In [574]:
ops = Operators()
ops.Synthesis([0,2,3,1,2],[2,3,0,0,2])

[2, 2, 0, 0, 2]

## III. IMCRO and MYIMCRO class

In [559]:
# Iteration
class IMCRO():
    optimal : MoleCule = None
    moleColl = 0.2
    alpha = random.randint(10, 100)
    beta = random.randint(10, 100)
    KELossRate = 0.6
    init_ke = 100
    buffer = 0
    pop : List[MoleCule]=[]
    ops : Operators = None
    interation = 1000
    '''
        pop : popsize the List Of Molecule Instance
        ops : for loa
        optimal : This attribute will hold the Molecule Instance with the lowest PE
        Which is also the output of this algorithm
    '''

    def __init__(self,pop : List[MoleCule]) -> None:
        '''
            Initialize the algorithm with this constructor 
            The parameter (Or the input) is the popsize which mean a List of Instance of Molecule Class
        '''
        self.pop=pop
        self.ops = Operators() # load in the operator for supersequence attribute of the molecule instance

        for mol in self.pop:
            '''
                The fit_func function is self-defined based on the data
                In this example the PE formula is defined as math.sin(m.structure[0]) + math.cos(m.structure[1]) 
                Which i have deveried from this class
            '''
            mol.pe = self.fit_func(mol)
            mol.ke = self.init_ke
            mol.update()
            if self.optimal is None:
                self.optimal = copy.deepcopy(mol)
            elif len(mol.supersequence) < len(self.optimal.supersequence):
                self.optimal = copy.deepcopy(mol)
    
    def run(self) -> None:
        start_time22 = time.time()
        '''
            The Algorithm starts here
            randomly pick which reaction uni(randomly take 1 molecule from popsize) 
            or inter (randomly take 2 molecule from popsize) 

            The number of iteration is abritrary 
        '''
        i=0
        while i!=self.interation:
            i+=1
            t = random.random()
            if t > self.moleColl or len(self.pop) < 2:
                self.uni_moleReact()
            else :
                self.inter_moleReact()
        end_time22 = time.time()  
        elapsed_time22 = end_time22 - start_time22
        print(f"Elapsed Time: {elapsed_time22} seconds")
    
    def is_subsequence(sub, main) -> bool:
        it = iter(main)
        return all(any(item == x for x in it) for item in sub)

    def is_supersequence(seq : List[int]) -> bool:
        subsequences = [[0, 1, 2], [1, 0, 3], [2, 3, 3], [3, 2, 1]]
        for i in subsequences:
            if not is_subsequence(i,seq):
                return False
        return True
    
    def update(self, m : MoleCule) -> None:
        """
        If m is the current optimal solution, save it to the optimal.
        """
        if len(m.supersequence) < len(self.optimal.supersequence) and is_supersequence(m.supersequence):
            self.optimal = copy.deepcopy(m)

    def uni_moleReact(self) -> None:
        m = random.choice(self.pop)
        if m.num_of_hits > self.alpha:
            self.decomposition(m)
        else :
            self.on_wall(m)

    def inter_moleReact(self) -> None:
        m1 , m2 = random.sample(self.pop,2)
        if m1.ke <= self.beta and m2.ke <= self.beta:
            self.synthesis(m1,m2)
        else :
            self.interaction(m1,m2)
    
    def decomposition(self,m : MoleCule) -> None:
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.dec(m)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m.pe + m.ke - new1.pe - new2.pe
        if tmp >= 0 or tmp + self.buffer >= 0:
            if tmp >= 0:
                q = random.random()
                new1.ke = tmp * q
                new2.ke = tmp * (1 - q)
            else:
                new1.ke = (tmp + self.buffer) * random.random() * random.random()
                new2.ke = (tmp + self.buffer - new1.ke) * random.random() * random.random()
                self.buffer = self.buffer + tmp - new1.ke - new2.ke
            new1.update()
            new2.update()
            self.pop.remove(m)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)
    def on_wall(self, m : MoleCule) -> None:
        m.num_of_hits += 1
        # You should implement this function in your derived class
        new = self.wall(m)
        new.pe = self.fit_func(new)
        tmp = m.pe + m.ke - new.pe
        if tmp >= 0:
            q = random.uniform(self.KELossRate, 1)
            new.ke = tmp * q
            new.update()
            self.buffer += tmp * (1 - q)
            self.pop.remove(m)
            self.pop.append(new)
            self.update(new)
    def interaction(self, m1 : MoleCule, m2 : MoleCule) -> None:
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.inter(m1, m2)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new1.pe - new2.pe
        if tmp >= 0:
            q = random.random()
            new1.ke = tmp * q
            new2.ke = tmp * (1 - q)
            new1.update()
            new2.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def synthesis(self, m1 : MoleCule, m2 : MoleCule) -> None:
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.syn(m1, m2)
        new.pe = self.fit_func(new)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new.pe
        if tmp >= 0:
            new.ke = tmp
            new.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new)
            self.update(new)

In [560]:
class MYIMCRO(IMCRO):
    def __init__(self, pop) -> None:
        super().__init__(pop)



    def fit_func(self, m : MoleCule):
            '''
            fit_func function for deciding PE base on the struct attribute
                     
            '''
            return math.sin(m.structure[0]) + math.cos(m.structure[1])
    


    def dec(self, m : MoleCule) -> List[MoleCule]:
        new1 = copy.deepcopy(m)
        new2 = copy.deepcopy(m)
        new1.supersequence,new2.supersequence = self.ops.Decomposition(m.supersequence)
        new1.structure[0] += random.random()
        new2.structure[1] += random.random()
        return [new1, new2]
    
    def wall(self, m : MoleCule) -> MoleCule:
        new = copy.deepcopy(m)
        new.structure[0], new.structure[1] = new.structure[1], new.structure[0]
        new.supersequence = self.ops.OnWall(m.supersequence)
        return new
    
    def inter(self, m1 : MoleCule, m2 : MoleCule) -> Tuple[str,str]:
        new1 = copy.deepcopy(m1)
        new2 = copy.deepcopy(m2)
        new1.supersequence,new2.supersequence = self.ops.Intermolecular(m1.supersequence,m2.supersequence)
        new1.structure[0] = m2.structure[0]
        new1.structure[1] = m1.structure[1]
        new2.structure[0] = m1.structure[0]
        new2.structure[1] = m2.structure[1]
        return [new1, new2]
    def syn(self, m1 : MoleCule, m2 : MoleCule) -> MoleCule:
        new = copy.deepcopy(m1)
        new.supersequence = self.ops.Synthesis(m1.supersequence,m2.supersequence)
        if random.random() < 0.5:
            new.structure[0] = m2.structure[0]
        else:
            new.structure[1] = m2.structure[1]
        return new

## IV. Run Code

In [561]:
initial = initialization(20,['acg', 'cat', 'gtt','tgc'])
for i in initial:
    print(len(i.supersequence))
myimcro = MYIMCRO(initial)
myimcro.run()

print(len(myimcro.optimal.supersequence))
print(myimcro.optimal.supersequence)

Population Generation:
['agcattgtgc', 'gtatcgatgc', 'gttacgattgc', 'catgactcgtt', 'actcgattgtgc', 'gtcgtcatcgat', 'acatgctt', 'caagtctgttgc', 'actggatctt', 'caagtcgttgc', 'cgatactgctg', 'acctgagttct', 'ctgactcggtt', 'atggccgctat', 'catgtgctgtc', 'tagcgcttact', 'tgacgtatt', 'acgctagctt', 'agtcgtctgcat', 'tgcagccatgtt']
Encodeding:
['0210332321', '2303120321', '23301203321', '10320131233', '013120332321', '231231031203', '01032133', '100231323321', '0132203133', '10023123321', '12030132132', '01132023313', '13201312233', '03221121303', '10323213231', '30212133013', '320123033', '0121302133', '023123132103', '321021103233']
10
10
11
11
12
12
8
12
10
11
11
11
11
11
11
11
9
10
12
12
Elapsed Time: 0.0210111141204834 seconds
8
[0, 1, 0, 3, 2, 1, 3, 3]


# ACB

## Functions
   - frequency
   - decode_to_int_list
   - check_all_elements

In [562]:
start_time1 = time.time()
def frequency(molecule: list) -> list:
    """
    OBject: Calculates the frequency of each element in the given molecule.

    Input:
        molecule (list<int>): List of atoms forming the molecule.

    Output:
        A list <int> representing the sorted frequencies of each element in the molecule.
    """
    array1 = {}
    for symbol in molecule:
        if symbol not in array1:
            array1[symbol] = 0
        array1[symbol] += 1

    my_keys = list(array1.keys())
    my_keys.sort()
    sorted_frequencies = {i: array1[i] for i in my_keys}

    result_array = [key for key, value in sorted_frequencies.items() for _ in range(value)]
    return result_array

def decode_to_int_list(encoded_population: list) -> list:
    """
    OBject: Decodes an encoded population to a list of integers.

    Input:
        encoded_population (list<string>): List of encoded strings.

    Output:
        A list <int> decoded from the given encoded population.
    """
    int_list = [int(char) for encoded_str in encoded_population for char in encoded_str]
    return int_list

def check_all_elements(my_list: list):
    """
    OBject: Checks if all specified elements are present in the given list.

    Input:
        my_list (list): The list to be checked.

    Output:
        True if all elements are present, False otherwise.
    """
    elements_to_check = [0, 1, 2, 3]
    return all(element in my_list for element in elements_to_check)

## Molecule_Bee and ArtificialBeeColony class

In [563]:
class Molecule_Bee:
    def __init__(self, molecule: list, frequencies: list):
        """
        OBject: Represents a molecule with a structure composed of atoms and associated frequencies.
        
        Attributes:
            molecule (list): List of atoms forming the molecule.
            frequencies (list): List of frequencies corresponding to each atom in the molecule.
            structure (list): Initial structure of the molecule, randomly chosen from the given atoms.
        """
        self.molecule = molecule
        self.frequencies = frequencies
        self.structure = [random.choice(molecule) for _ in range(len(molecule))] 

    def calculate_fitness(self):
        """
        OBject: Calculates the fitness of the current molecule structure based on frequencies.

        Output:
            The fitness value: int
        """
        fitness = 0
        for atom in self.structure:
            fitness += self.frequencies[self.molecule.index(atom)]
        return fitness

    def generate_new_structure(self):
        """
        OBject: Generates a new random structure for the molecule based on frequencies.

        Output:
            A new structure (list) for the molecule composed of randomly chosen frequencies.
        """
        new_structure = [random.choice(self.frequencies) for _ in range(len(self.frequencies))]
        return new_structure

class ArtificialBeeColony:
    def __init__(self, initial_pop: list, frequencies: list, population_size: int, n, max_cycles: int):
        """
        OBject: Represents an Artificial Bee Colony optimization algorithm for molecule structure.
        
        Attributes:
            initial_pop: Initial population for the optimization.
            molecule (list): List of atoms forming the molecule. (Note: molecule is not defined in this scope.)
            frequencies (list): List of frequencies corresponding to each atom in the molecule.
            population_size (int): Size of the population in the optimization.
            max_cycles (int): Maximum number of cycles or iterations.
            mo: Initialization (Assumed to be a function or data structure) result for the population.
            n (int): Number of cycles or iterations.
            molecules (list): List of Molecule_Bee instances representing the population of molecules.
        """
        self.initial_pop = initial_pop
        self.molecule = molecule
        self.frequencies = frequencies
        self.population_size = population_size
        self.max_cycles = max_cycles
        self.mo = initialization(population_size,initial_pop)
        self.n = n
        
        self.molecules = [Molecule_Bee(self.mo[i].getSuper(), self.frequencies) for i in range(population_size)]
        


    def solve(self) -> list:
        """
        OBject: Solves the optimization problem using the Artificial Bee Colony algorithm.

        Output:
            The best structure(list<int>) found by the algorithm.
        """
        start_time1 = time.time() #Start
        
        best_structure = self.molecules[0].structure.copy()
        best_fitness = self.calculate_fitness(best_structure)

        for cycle in range(self.max_cycles):
            # Evaluate fitness and update the best structure
            for molecule in self.molecules:
                fitness = molecule.calculate_fitness()
                # Update best structure if fitness is higher and all elements are present in the molecule
                if fitness > best_fitness & check_all_elements(molecule.molecule):
                    best_structure = molecule.structure.copy()
                    best_fitness = fitness

            average_fitness = self.average_fitness()
            # Employ artificial bee algorithm to generate new structures
            for molecule in self.molecules:
                if molecule.calculate_fitness() < average_fitness:
                    new_structure = molecule.generate_new_structure()
                    new_fitness = self.calculate_fitness(new_structure)
                    # Accept the new structure if it improves fitness
                    if new_fitness > molecule.calculate_fitness():
                        molecule.structure = new_structure.copy()
        end_time = time.time()  # End
        elapsed_time = end_time - start_time1
        print(f"Elapsed Time: {elapsed_time} seconds")
        return best_structure

    def calculate_fitness(self, structure: list) -> int:
        """
        OBject: Calculates the fitness of a given structure.

        Input:
            structure (list): The structure for which fitness needs to be calculated.

        Output:
            The fitness value of the given structure.
        """
        fitness = 0
        for atom in structure:
            fitness += self.frequencies[self.molecule.index(atom)]
        return fitness

    def average_fitness(self) -> float:
        """
        OBject: Calculates the average fitness of all molecules in the population.

        Output:
            The average fitness(float) value of the population.
        """
        total_fitness = sum(molecule.calculate_fitness() for molecule in self.molecules)
        return total_fitness / self.n


## Run Code 

In [564]:

pop_size = 20
initial_population = ['acg', 'cat', 'gtt', 'tgc']
encoded_population = encoding_population(initial_population)
molecule = decode_to_int_list(encoded_population)

# print(molecule)
n = len(initial_population)
frequencies = frequency(molecule)
max_cycles=1000

molecule_colony = ArtificialBeeColony(initial_population,frequencies, pop_size, n, max_cycles)
best_structure = molecule_colony.solve()

print("Best Structure for ACB :", best_structure)


Population Generation:
['cgttagctg', 'taggctcatg', 'acgtcatgtgct', 'caatgccgtt', 'atgcgattt', 'gtcatctgcat', 'accgatgctt', 'gttatcgcactg', 'gatctgtcacgt', 'atgtgcgtatc', 'ctgacatgtt', 'tgacgtctcatg', 'gatctgactg', 'tgacattggct', 'gcttacgttgc', 'tacggcgatt', 'gatcgtactg', 'cgatatctgtgc', 'agttccgctgat', 'tcagcctgtt']
Encodeding:
['123302132', '3022131032', '012310323213', '1003211233', '032120333', '23103132103', '0112032133', '233031210132', '203132310123', '03232123031', '1320103233', '320123131032', '2031320132', '32010332213', '21330123321', '3012212033', '2031230132', '120303132321', '023311213203', '3102113233']
Elapsed Time: 0.17307472229003906 seconds
Best Structure for ACB : [2, 3, 2, 3, 1, 2, 1, 2, 3, 0, 2, 1]


# ACO

In [565]:
start_time23 = time.time()
def initialization2(pop_size, set_of_strings):
    
    """
    Initialize the population for the SCS problem.

    Parameters:
    - pop_size (int): The size of the population.
    - set_of_strings (list): A list of strings to be combined into supersequences.

    Returns:
    - list: The encoded population for the SCS problem.
    """
    initial_population = population_generation(pop_size, set_of_strings)
    encoded_population = encoding_population(initial_population)

    return encoded_population

In [566]:
"""
The Ant Colony Optimization (ACO) algorithm simulates the foraging behavior of ants to find an optimal path.
In this algorithm, a predefined number of ants explore possible paths, leaving behind a pheromone trail.
The intensity of the pheromone trail is influenced by the number of ants that traverse a particular path.
Paths with a higher concentration of pheromones become more attractive to subsequent ants, increasing the likelihood of those paths being chosen as optimal routes.
For instance, if multiple ants traverse a path with different distances, the algorithm calculates the total
pheromone level on that path, reflecting the cumulative choices of all ants that have moved along it. This collective information guides the algorithm in identifying promising paths for the final solution.
Approach to Solve SCS Problem using ACO Algorithm:

Step 1: Encode Subsequences
- Encode each subsequence as a number (0 to 3), representing different elements.

Step 2: Create 2D List
- Form a 2D list where each row corresponds to the string value of an encoded subsequence.

Step 3: Apply ACO Algorithm
- Utilize the Ant Colony Optimization algorithm on the 2D list, treating each row as a node.
- Find the most optimal path through these nodes, representing a solution to the SCS problem.

Step 4: Translate Optimal Path
- Translate the obtained optimal path back into the original sequence of 0 to 3.

Step 5: Output Optimal SCS
- Construct the most optimal Shortest Common Supersequence (SCS) using the translated path.
"""
def ACO(initial_pop, popsize,iteration):
    
    
    
    molecules = []  # Initialize an empty list for storing encoded subsequences.
    origin = initialization2(popsize, initial_pop)


    # Initialize a list of subsequences that are encoded based on the encoding_population function.
    for original_string in origin:
        digit_list = [int(digit) for digit in original_string]
        molecules.append(digit_list)
    flattened_molecules = [item for sublist in molecules for item in sublist]

    #unique number in the molecules
    unique_elements = list(set(flattened_molecules))
    unique_elements.sort()
    # FILP the matrix so that it fits the condition that the column number is larger than the row number
    distance_matrix =list(map(list, zip(*molecules)))
    # for m in distance_matrix :
    #     print(m)
    # print(len(distance_matrix[0]))
    n = len(distance_matrix)

    # sumarize the total cost walk by an ant in the given path
    # ant_path : list<int>
    def calculate_distance(ant_path: list) -> float:

        current_index = ant_path[0]  # Calculation of distance between nodes
        distance = 0
        for next_index in ant_path[1:]:
            # print(current_index)
            # print(next_index)
            distance += distance_matrix[current_index][next_index]
            current_index = next_index
        return distance  # Distance returned

    # swap a given sequence
    # sequence :list<int>
    def swap(sequence: list, i: int, j: int) -> None:

        temp = sequence[i]  # Node swapping function
        sequence[i] = sequence[j]
        sequence[j] = temp

    # update a single path that the ant have walk then return the updated calculate_distance
    # ant_path :tuple(list<int> , int)
    def local_pheromone_update(ant_path: tuple, a: int, b: int) -> tuple:

        updated_ant_path = ant_path[0][:]
        swap(updated_ant_path, a, b)
        return (updated_ant_path, calculate_distance(updated_ant_path))  # Return the ant path.
    # update multiple path that the ant have walk then return the updated calculate_distance
    # ant_path :tuple(list<int> , int)
    def global_pheromone_update(ant_path: tuple, a: int, b: int, c: int) -> tuple:
        updated_ant_path = ant_path[0][:]
        swap(updated_ant_path, a, b)
        swap(updated_ant_path, b, c)
        return (updated_ant_path, calculate_distance(updated_ant_path))  # Return the ant path.


    while True:
        num_ants = 10  # Number of ants
        # worst path or good path or determin by a poprebility if an worst ant have a hige chances of taking a path with low pop while good ant do the oppersite
        worst_ants = int(0.1 * num_ants)  # ant that take a worst path

        best_ants = int(0.9 * num_ants)  # Good-value ants  , ant that take a good path(less costly)

        # alpha and beta are just constance that can  be change to see different result
        alpha = 2.0  # Alpha value required for the transition method (float)
        beta = 2.0  # Beta value required for the transition method (float)
        pass_max = 15  # Transition method variables (increased pass_max for a higher range)
        pass_min = 0  # Transition method variables
        transition_probability = 0.5  # Float value the poperties in which the ants will transition
        pass_method = alpha * 1 / n * beta * (pass_max - pass_min)  # Float value
        iteration_size = iteration * 120  # How many times the main loop will run
        ants = []  # Array (list) for ants
        initial_path = list(range(0, n))
        for i in range(num_ants):
            # Generate the initial path, meaning that it randomly generates path, order in which each row is selected
            ant_path = sample(initial_path, n)
            # print(ant_path)
            ants.append((ant_path, calculate_distance(ant_path)))

        # Sort the second element of the structure, i.e., the distance in which the ant will travel
        ants.sort(key=lambda x: x[1])
        # The main loop in the program
        for iteration_index in range(iteration_size):
            go_ant = ants[randint(0, best_ants)]  # Movement group to be selected
            random_ant_index = randint(0, int(pass_method))  # The next group will be shaped according to the transition method
            # The transition probability , the probability in which an ant will choose a path
            # the random.random function is an number between [0,1] it decide if the ant sort change it operation
            if random.random()  < transition_probability:

                more_powerful_ant = local_pheromone_update(go_ant, randint(0, n-1), randint(0, n-1))
                if ants[random_ant_index][1] > more_powerful_ant[1]:
                    ants[random_ant_index] = more_powerful_ant
            else:
                for i in range(num_ants - worst_ants, num_ants):
                    ants[i] = local_pheromone_update(ants[i], randint(0, n-1), randint(0, n-1))
                ants.sort(key=lambda x: x[1])

            # get the ant with the lowest cost ie the path that the ant walk with the lowest cost
            low_cost_ant = ants[0]
            effectively_global_ant = global_pheromone_update(
                low_cost_ant, randint(0, n-1), randint(0, n-1), randint(0, n-1))
            # make a goble pheromon udpate so that it can check again if the lowest cost ant that have global_pheromone_update distince is smaller than the ant which does not
            if ants[0][1] > effectively_global_ant[1]:
                ants[0] = effectively_global_ant
            ants.sort(key=lambda x: x[1])
        # Variable to store the cost that the ant will take
        ant_costs = []
        # For loop for adding the cost that the ant will take
        for i in range(len(ants[0][0]) - 1):
            current_city = ants[0][0][i]
            next_city = ants[0][0][i + 1]
            cost = molecules[current_city][next_city]
            ant_costs.append(cost)
        unique_elements = set(ant_costs)
        # Condition to break if the ant has taken a path with all encoded variable words in that are contained in the initialization, i.e., ['acg', 'cat', 'gtt','tgc']
        if set([0,1,2,3]).issubset(unique_elements):
            
            end_time23 = time.time()  # Ghi lại thời điểm kết thúc giải thuật
            elapsed_time23 = end_time23 - start_time23
            print(f"Elapsed Time: {elapsed_time23} seconds")
            
            return ant_costs

## Run Code

In [567]:
ant_costs = ACO(['acg', 'cat', 'gtt', 'tgc'], 100, 1000)
print("SCS for ACO", ant_costs)

Elapsed Time: 3.682814359664917 seconds
SCS for ACO [3, 2, 1, 2, 1, 0, 1]


# Compare Time

In [568]:
def compareTime(initial_pop: List[str], pop_size: List[int], iteration: List[int]):
    result = []
    ## IMCRO
    for i in range(len(pop_size)):
        timeRun = []
        start_time1 = time.time()
        initial = initialization(pop_size[i],initial_pop)
        myimcro1 = MYIMCRO(initial)
        myimcro1.run()
        end_time1 = time.time()
        duration1 = end_time1 - start_time1
        
        timeRun.append(duration1)
        
        ## ACB
        start_time2 = time.time()
        encoded_population = encoding_population(initial_pop)
        molecule = decode_to_int_list(encoded_population)

        # print(molecule)
        n = len(initial_pop)
        frequencies = frequency(molecule)

        molecule_colony = ArtificialBeeColony(initial_pop,frequencies, pop_size[i], n, iteration[i])
        best_structure = molecule_colony.solve()
        
        end_time2 = time.time()
        duration2 = end_time2 - start_time2
        timeRun.append(duration2)
        ## ACO
        start_time3 = time.time()
        ant_costs = ACO(initial_pop, pop_size[i], iteration[i])
        end_time3 = time.time()
        duration3 = end_time3 - start_time3
        timeRun.append(duration3)
        result.append(timeRun)
    
    return result
    
     

In [569]:
initial_pop1 = ['acg', 'cat', 'gtt','tgc']
pop_size1 = [20, 50, 100, 500]
iteration = [10, 100, 200, 500]


In [570]:
time = compareTime(initial_pop1, pop_size1, iteration)

Population Generation:
['cagttcagtgc', 'caacgttggc', 'acgcgtattgc', 'gttaccgtatgc', 'gcactttgtgc', 'gttcgactacg', 'tgacctcgatt', 'tgcatcgat', 'gactgctagtc', 'ctaggactgt', 'gactcgtattgc', 'taggccttgat', 'tcatgcttacg', 'acatgtgtgtc', 'gactatcttggc', 'cgaatcttggc', 'agctgcgtattc', 'gctatgccttg', 'catgacttggc', 'tgcacgagtt']
Encodeding:
['10233102321', '1001233221', '01212303321', '233011230321', '21013332321', '23312013012', '32011312033', '321031203', '20132130231', '1302201323', '201312303321', '30221133203', '31032133012', '01032323231', '201303133221', '12003133221', '021321230331', '21303211332', '10320133221', '3210120233']
Elapsed Time: 0.019022226333618164 seconds
Population Generation:
['tgcagcgctat', 'ctagatcgtgtc', 'tacgtcctat', 'caatggtctcg', 'tcgcactgtt', 'tgacggcattt', 'agcattttggc', 'tgacgattcg', 'gcattcgtgc', 'gtagcgcatc', 'cgatttgccg', 'gactttcgcat', 'gtatccagttgc', 'tgttagccgcat', 'gcatcgttgc', 'agtctatgcgtc', 'gcttactggact', 'acggttcgcat', 'gttacgatgc', 'cagactgttgc']
E

# Result

In [571]:
time

[[0.019022226333618164, 0.0019998550415039062, 0.007097959518432617],
 [0.020998477935791016, 0.04401373863220215, 0.0635528564453125],
 [0.021998167037963867, 0.1750960350036621, 0.12054610252380371],
 [0.15111017227172852, 2.2468137741088867, 0.31064891815185547]]